# Diving into French Presidential Discourses
## Yannick Jadot (YJ)

In [ ]:
!pip install minet
!pip install pdfminer.six
!pip install unidecode
!python -m spacy download fr_core_news_sm

In [2]:
import pandas as pd
from pdfminer.high_level import extract_text
from gensim.test.utils import datapath
from gensim import utils
import spacy
from collections import Counter
import unidecode
import gensim.models

In [3]:
## create a .csv of YJ's Twitter activity since 27 Jan 2022
!minet tw scrape tweets "(from:yjadot) until:2022-03-27 since:2022-01-27" > tweets_YJ.csv

## convert the .csv file in a data frame using pandas
df_tw_YJ = pd.read_csv("./tweets_YJ.csv")

## create a list of tweets selecting the 'text' column of the data frame
list_tw_YJ = df_tw_YJ['text'].values.tolist()
len(list_tw_YJ)

Searching for "(from:yjadot) until:2022-03-27 since:2022-01-27"


855

In [4]:
## retrieve YJ's affiliates1 Twitter activity
!minet tw scrape tweets "(from:_EvaSas OR from:julienbayou OR from:sandraregol OR from:iordanoff OR from:CoulombelAlain OR from:FThiollet OR from:marinetondelier OR from:Csagaspe OR from:delphinebatho OR from:EricPiolle OR from:hub_laferriere OR from:SabrinaSebaihi OR from:Melanie_Vogel_ OR from:MounirSatouri) until:2022-03-27 since:2022-01-27" > tw_YJ_aff_all.csv

## convert YJ's affiliates' tweets in a list
df_tw_YJ_aff_all = pd.read_csv("tw_YJ_aff_all.csv")
list_tw_YJ_aff_all = df_tw_YJ_aff_all['text'].values.tolist()
print(list_tw_YJ_aff_all[0])
print(len(list_tw_YJ_aff_all))

Searching for "(from:_EvaSas OR from:julienbayou OR from:sandraregol OR from:iordanoff OR from:CoulombelAlain OR from:FThiollet OR from:marinetondelier OR from:Csagaspe OR from:delphinebatho OR from:EricPiolle OR from:hub_laferriere OR from:SabrinaSebaihi OR from:Melanie_Vogel_ OR from:MounirSatouri) until:2022-03-27 since:2022-01-27"
🗣 « Ne vous laissez pas déposséder de cette élection !
Ne laisser pas l’enjeu du #climat disparaître de la #presidentielles2022 »

🌱@yjadot était l’invité du journal de @LaurentDelahous sur @France2tv 

🎥 le replay : https://youtu.be/R0O-Yjy9pu8 https://twitter.com/delphinebatho/status/1507835199505248258/photo/1
1965


In [5]:
## convert .csv affiliates' tweets in a list
df_tw_YJ_aff_all = pd.read_csv("tw_YJ_aff_all.csv")
list_tw_YJ_aff_all = df_tw_YJ_aff_all['text'].values.tolist()
print(list_tw_YJ_aff_all[0])
print(len(list_tw_YJ_aff_all))

🗣 « Ne vous laissez pas déposséder de cette élection !
Ne laisser pas l’enjeu du #climat disparaître de la #presidentielles2022 »

🌱@yjadot était l’invité du journal de @LaurentDelahous sur @France2tv 

🎥 le replay : https://youtu.be/R0O-Yjy9pu8 https://twitter.com/delphinebatho/status/1507835199505248258/photo/1
1965


In [6]:
## merge YJ's and YJ's affiliates lists
list_tw_YJ_all = list_tw_YJ + list_tw_YJ_aff_all

In [7]:
## i retrieve a string from the pdf of YJ's manifesto using extract_text of the pdfminer package
## the cleaning process is specific for this manifesto and it depends on the output of extract_text
manif_YJ = extract_text('/Users/simonemariaparazzoli/Documents/Università/Sciences Po/Diving into public digital spaces/research/manifesto_jadot.pdf')
manif_clean_YJ = manif_YJ.replace(' .','')
manif_clean_YJ = manif_clean_YJ.replace('   ','')
manif_clean_YJ = manif_clean_YJ.replace('\n\n','')
manif_clean_YJ = manif_clean_YJ.replace('\n','')
manif_clean_YJ = manif_clean_YJ.replace('. ','---')
manif_clean_YJ = manif_clean_YJ.replace('\x0c',' ')
#print(repr(manif_clean_JLM))

## convert the string of the manifesto into a list
list_manif_YJ = manif_clean_YJ.split("---")
list_manif_YJ = [s for s in list_manif_YJ if len(s)>20]
len(list_manif_YJ)
#print(list_manif_EM)'''

365

In [18]:
## merge the two lists of tweets and of the manifesto 
list_YJ = list_tw_YJ_all + list_tw_YJ_all + list_manif_YJ
len(list_YJ)

6005

In [19]:
## load a spacy model to retrieve stop words
nlp = spacy.load("fr_core_news_sm")
stop_words_fr = nlp.Defaults.stop_words
#new_sw = ["avec","la","les","le","pour","un","une","nous","ete","et","je"]
#stop_words_fr.add(new_sw)

## clean the list of tweets and manifesto to get rid of useless words and make the list content homogeneous
list_YJ_clean = []
for i in list_YJ:
    doc = nlp(i)
    tokens = [unidecode.unidecode(token.text).lower()for token in doc 
              if ( token.text not in stop_words_fr and
                  len(token.text)>1 and
                  token.like_url == False )]
    tokens_joined = ' '.join(tokens)
    list_YJ_clean.append(tokens_joined)

In [20]:
## test the output of the cleaning process
print(list_YJ[2401])
print("---")
print(list_YJ_clean[2401])

Le scandale continue : Manuel Domergue : "On a 2,2 millions de ménages en attente de logement s... https://youtu.be/XvBydU-Ip_I via @YouTube
---
le scandale continue manuel domergue on 2,2 millions menages attente logement ... @youtube


In [21]:
## prepare the corpus as a class
class MyCorpus_YJ:

    def __iter__(self):
        for i in list_YJ_clean:
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(i,min_len=3)

In [78]:
## train the word embeddings model_YJ
sentences = MyCorpus_YJ()
model_YJ = gensim.models.Word2Vec(sentences=sentences, min_count=10, vector_size=300, epochs=100)

In [52]:
## transform the corpus list (that is made of tweets and sentences from the manifesto)
## in a list containing all the words of the corpus as elements of the list
words_YJ = []

for i in list_YJ_clean:
    i_split = i.split(' ') #transform the i document into a list (split at blank space)
    words_YJ.extend(i_split)

## clean the list of tokens
words_YJ_clean = [x for x in words_YJ 
                   if x not in stop_words_fr
                   if x != "\n\n"
                   if len(x)>1]

In [53]:
## find the 30 most common words using Counter
words_freq_YJ = Counter(words_YJ_clean)
common_words_YJ = words_freq_YJ.most_common(30)
print(common_words_YJ)

[('jadot2022', 2080), ('changeons', 604), ('@yjadot', 582), ('france', 515), ('climat', 383), ('ukraine', 382), ('contre', 348), ('faire', 323), ("aujourd'hui", 322), ('democratie', 313), ('guerre', 306), ('faut', 297), ('macron', 296), ('ans', 288), ('poutine', 284), ('politique', 281), ('russie', 280), ('faireface', 270), ('...', 252), ('francais', 246), ('projet', 244), ('ecologistes', 244), ('climatique', 242), ('elysee2022', 242), ('femmes', 238), ('pays', 231), ('justice', 226), ('ecologie', 225), ('europe', 217), ('ete', 215)]


In [54]:
## first attempt with the most_similar function on our corpus using our model_YJ
result = model_YJ.wv.most_similar(positive=['societe'], topn=30)
print(result)

[('solidaire', 0.3697938323020935), ('epanouissement', 0.367585688829422), ('enrichis', 0.3561038672924042), ('reparation', 0.34013983607292175), ('reparer', 0.33930259943008423), ('attendre', 0.32666659355163574), ('chers', 0.3224840760231018), ('ecole', 0.3206920623779297), ('envie', 0.31748467683792114), ('choisir', 0.3158099055290222), ('montee', 0.31053388118743896), ('constance', 0.3091202974319458), ('maltraite', 0.3085594177246094), ('humanistes', 0.30800527334213257), ('choix', 0.30488860607147217), ('transformer', 0.30214542150497437), ('comprendre', 0.29967543482780457), ('amis', 0.2960008978843689), ('methodes', 0.28936856985092163), ('accompagnerons', 0.28737327456474304), ('accessible', 0.2821170687675476), ('adapter', 0.2810925543308258), ('partage', 0.28093916177749634), ('mobiliser', 0.27699053287506104), ('modele', 0.27672404050827026), ('feministe', 0.27355435490608215), ('abiment', 0.27345219254493713), ('valeur', 0.2732841372489929), ('comportements', 0.27007213234

In [55]:
print(len(list_tw_YJ_all),',',len(list_YJ_clean))

2820 , 6005


In [79]:
result = model_YJ.wv.most_similar(positive=['france'], topn=20)
print(result)

[('internationaux', 0.2397458255290985), ('joue', 0.2220449596643448), ('dictateurs', 0.21624337136745453), ('amis', 0.21273811161518097), ('chers', 0.2091188281774521), ('asile', 0.2084098756313324), ('jadot', 0.206181600689888), ('humanistes', 0.19934490323066711), ('avenir', 0.19821365177631378), ('participer', 0.1945587545633316), ('mali', 0.18484660983085632), ('ukrainien', 0.1833828091621399), ('accord', 0.18022622168064117), ('trouve', 0.17901581525802612), ('europe', 0.17898990213871002), ('menacent', 0.17738595604896545), ('accueillir', 0.1756560206413269), ('depute', 0.17528210580348969), ('procedure', 0.17388492822647095), ('agriculteurs', 0.1723768413066864)]


In [57]:
result = model_YJ.wv.most_similar(positive=['etat'], topn=20)
print(result)

[('separation', 0.3720186948776245), ('agences', 0.3552718460559845), ('etanche', 0.3404220938682556), ('gratuite', 0.3266077935695648), ('lobbys', 0.31149712204933167), ('criminelle', 0.30876103043556213), ('sorte', 0.3002417981624603), ('librement', 0.2969236373901367), ('coulisses', 0.2909714877605438), ('laolp', 0.28489813208580017), ('confiance', 0.28196051716804504), ('reprendre', 0.28194659948349), ('corse', 0.281080037355423), ('ami', 0.2803851068019867), ('cerf', 0.2713429629802704), ('pouvoirs', 0.2676903307437897), ('montagnedor', 0.26637670397758484), ('petite', 0.26403990387916565), ('barriere', 0.26218748092651367), ('libre', 0.2599906921386719)]


In [58]:
result = model_YJ.wv.most_similar(positive=['souverainete'], topn=20)
print(result)

[('alimentaire', 0.7082191109657288), ('impacts', 0.3849615752696991), ('contribuent', 0.35720035433769226), ('puissance', 0.3565831184387207), ('jadotmarseille', 0.35497918725013733), ('triptyque', 0.35350048542022705), ('independance', 0.35180744528770447), ('protege', 0.33877626061439514), ('engageons', 0.30807703733444214), ('slava', 0.30786076188087463), ('aveugle', 0.30738329887390137), ('essentiel', 0.3066161870956421), ('rappeler', 0.30539625883102417), ('fossiles', 0.3026944398880005), ('activiste', 0.30238449573516846), ('dependances', 0.30173078179359436), ('autonomie', 0.3009205162525177), ('conflit', 0.2959980070590973), ('outremer', 0.29307836294174194), ('energetique', 0.28898194432258606)]


In [59]:
result = model_YJ.wv.most_similar(positive=['president'], topn=20)
print(result)

[('evoquer', 0.3501220643520355), ('apprendre', 0.3339003324508667), ('emmanuel', 0.3249945640563965), ('marleneschiappa', 0.313765287399292), ('raciste', 0.2993308901786804), ('exprimer', 0.2890198230743408), ('bilan', 0.2869185507297516), ('antisemite', 0.28127041459083557), ('officiellement', 0.2726139724254608), ('tours', 0.2686164677143097), ('respecte', 0.26373258233070374), ('autonome', 0.2575867176055908), ('madame', 0.25248366594314575), ('affirme', 0.25243791937828064), ('attendant', 0.24765032529830933), ('sportifs', 0.2448710799217224), ('denis', 0.24238130450248718), ('drapeau', 0.23912270367145538), ('candidate', 0.23849576711654663), ('presidentiel', 0.23550927639007568)]


In [60]:
result = model_YJ.wv.most_similar(positive=['politique'], topn=20)
print(result)

[('attentes', 0.34851256012916565), ('fonctionnement', 0.3199769854545593), ('second', 0.28334304690361023), ('humaniste', 0.27200472354888916), ('eloigne', 0.26117846369743347), ('ggrmc', 0.25869879126548767), ('democratique', 0.25351041555404663), ('signe', 0.2523189187049866), ('attaque', 0.24922531843185425), ('puissance', 0.24754959344863892), ('union', 0.2405264675617218), ('promesse', 0.2385319620370865), ('presidence', 0.23796267807483673), ('conge', 0.23775537312030792), ('construit', 0.23379886150360107), ('mali', 0.23338381946086884), ('origine', 0.23079445958137512), ('integration', 0.22981871664524078), ('internationales', 0.22801728546619415), ('vivre', 0.2277868539094925)]


In [61]:
result = model_YJ.wv.most_similar(positive=['droit'], topn=20)
print(result)

[('ivg', 0.41173407435417175), ('avortement', 0.3783719539642334), ('danger', 0.3676586449146271), ('fondamental', 0.35687732696533203), ('violation', 0.35475146770477295), ('filles', 0.3477737307548523), ('familiales', 0.33727824687957764), ('expression', 0.33486682176589966), ('protege', 0.3224516212940216), ('institutions', 0.31922194361686707), ('souhaitent', 0.30892932415008545), ('jouer', 0.3070260286331177), ('principe', 0.30661946535110474), ('territoriale', 0.29901668429374695), ('aggression', 0.2937557101249695), ('reelle', 0.29360488057136536), ('croire', 0.2902195453643799), ('opinion', 0.28846436738967896), ('quelques', 0.2880440056324005), ('forte', 0.28674906492233276)]


In [62]:
result = model_YJ.wv.most_similar(positive=['entreprise'], topn=20)
print(result)

[('billet', 0.40212133526802063), ('alsisiofficial', 0.36670559644699097), ('servir', 0.33407896757125854), ('innovation', 0.32841235399246216), ('accords', 0.3208222985267639), ('orienter', 0.30466195940971375), ('strategie', 0.3025156259536743), ('portrait', 0.29984787106513977), ('francaise', 0.2951377034187317), ('cihrs_alerts', 0.294289767742157), ('espace', 0.2930728495121002), ('precis', 0.2916492223739624), ('signature', 0.29053089022636414), ('sert', 0.28888648748397827), ('apaiser', 0.2873527705669403), ('diplomatie', 0.2860443592071533), ('role', 0.28548532724380493), ('reaction', 0.2852196991443634), ('articule', 0.2849511504173279), ('lancer', 0.2825457751750946)]


In [63]:
result = model_YJ.wv.most_similar(positive=['economie'], topn=20)
print(result)

[('robuste', 0.6122744679450989), ('circulaire', 0.58043372631073), ('resiliente', 0.5749534964561462), ('reorienter', 0.46580860018730164), ('phase', 0.4249250590801239), ('souveraine', 0.4207841753959656), ('horizon', 0.3914775252342224), ('confort', 0.3884466886520386), ('imperatifs', 0.37628087401390076), ('malade', 0.3615204095840454), ('instaurer', 0.3590092957019806), ('orientation', 0.35600531101226807), ('financiers', 0.3512500822544098), ('partage', 0.35092693567276), ('planete', 0.3474362790584564), ('achat', 0.33677804470062256), ('soit', 0.33381739258766174), ('axes', 0.32089921832084656), ('polluantes', 0.3177948296070099), ('partenariat', 0.31222885847091675)]


In [64]:
result = model_YJ.wv.most_similar(positive=['emploi'], topn=20)
print(result)

[('pole', 0.5063161849975586), ('bon', 0.40599682927131653), ('creer', 0.4021540582180023), ('emplois', 0.377839058637619), ('respectueuses', 0.345875084400177), ('surendettement', 0.3368813097476959), ('sobriete', 0.33096233010292053), ('local', 0.3294264078140259), ('pfue', 0.32505980134010315), ('chomage', 0.31773945689201355), ('facile', 0.3175324499607086), ('marche', 0.31733861565589905), ('handicape', 0.3127240836620331), ('dignement', 0.3119063377380371), ('cree', 0.3108161389827728), ('reflechir', 0.30556631088256836), ('logement', 0.3021449148654938), ('agroecologie', 0.30162662267684937), ('vivant', 0.30099761486053467), ('automobile', 0.29970651865005493)]


In [65]:
result = model_YJ.wv.most_similar(positive=['travail'], topn=20)
print(result)

[('mobilites', 0.398993581533432), ('durables', 0.34791049361228943), ('dignes', 0.3277805745601654), ('redonnerons', 0.31345334649086), ('porterons', 0.31044307351112366), ('long', 0.3055921196937561), ('uber', 0.3034379184246063), ('independants', 0.3026772439479828), ('soignants', 0.302579790353775), ('rentabilite', 0.3006685972213745), ('consacre', 0.29985612630844116), ('foot', 0.29783496260643005), ('drogues', 0.2872268259525299), ('hijabeuses', 0.2862699031829834), ('laissent', 0.28485891222953796), ('industrie', 0.28432518243789673), ('remunerations', 0.28173139691352844), ('remboursement', 0.28056952357292175), ('saluer', 0.27871057391166687), ('obligatoire', 0.27833953499794006)]


In [66]:
result = model_YJ.wv.most_similar(positive=['numerique'], topn=20)
print(result)

[('sobriete', 0.43591615557670593), ('personnelles', 0.43246129155158997), ('mettrai', 0.41871803998947144), ('normes', 0.38023972511291504), ('pac', 0.3789988160133362), ('filieres', 0.36779531836509705), ('devront', 0.367272287607193), ('fonctionne', 0.3669591248035431), ('charte', 0.36522620916366577), ('naturels', 0.3551957607269287), ('donnees', 0.35370245575904846), ('espaces', 0.3487623929977417), ('loyers', 0.34764012694358826), ('consultez', 0.3445090353488922), ('langues', 0.34444236755371094), ('pratiques', 0.3301425278186798), ('anticipation', 0.3266052305698395), ('presenter', 0.32273194193840027), ('respectueuse', 0.32252806425094604), ('sortirons', 0.3213912844657898)]


In [67]:
result = model_YJ.wv.most_similar(positive=['donnees'], topn=20)
print(result)

[('personnelles', 0.7087156772613525), ('reponses', 0.5234103202819824), ('travaux', 0.46023163199424744), ('pediatriques', 0.4237605929374695), ('nevers', 0.40710100531578064), ('echanges', 0.39264243841171265), ('twitt', 0.3904799818992615), ('injustement', 0.3889560103416443), ('presenter', 0.3796391189098358), ('articule', 0.3667319118976593), ('prevention', 0.3638712465763092), ('speculation', 0.3597400188446045), ('numerique', 0.35370245575904846), ('encourager', 0.35358843207359314), ('considerer', 0.3479935824871063), ('consideree', 0.3458349108695984), ('imaginaires', 0.3452266752719879), ('partages', 0.33913683891296387), ('educatives', 0.33441486954689026), ('carolinerooseeu', 0.3339465260505676)]


In [68]:
result = model_YJ.wv.most_similar(positive=['monde'], topn=20)
print(result)

[('futur', 0.3023635447025299), ('reformes', 0.28281471133232117), ('cegivors', 0.27230897545814514), ('modele', 0.2502337396144867), ('articule', 0.24895620346069336), ('electrique', 0.2481425255537033), ('gouvernants', 0.2460928112268448), ('ses', 0.23698356747627258), ('zaporizhzhia', 0.23181460797786713), ('meres', 0.23058739304542542), ('menacent', 0.22899296879768372), ('urgences', 0.22238975763320923), ('thermiques', 0.2162628024816513), ('forme', 0.21521906554698944), ('protegent', 0.21134816110134125), ('sortons', 0.2086954414844513), ('blah', 0.20843349397182465), ('agissant', 0.20761717855930328), ('chers', 0.20675909519195557), ('choisi', 0.2059941589832306)]


In [69]:
result = model_YJ.wv.most_similar(positive=['realite'], topn=20)
print(result)

[('dogme', 0.42001423239707947), ('regards', 0.396262526512146), ('inaction', 0.3864611089229584), ('oneoceansummit', 0.3804419934749603), ('brevets', 0.36417827010154724), ('bah', 0.35854077339172363), ('ouganda', 0.33379319310188293), ('amour', 0.32959750294685364), ('phase', 0.32920295000076294), ('chiffre', 0.3281971514225006), ('organise', 0.3221087157726288), ('beurfm', 0.3220147490501404), ('parquets', 0.3211840093135834), ('acceleration', 0.3126322329044342), ('desaccords', 0.3123561441898346), ('gros', 0.308511346578598), ('brest', 0.30741655826568604), ('exposee', 0.295977920293808), ('peche', 0.2952602505683899), ('imperatifs', 0.29414913058280945)]


In [70]:
result = model_YJ.wv.most_similar(positive=['verite'], topn=20)
print(result)

[('battent', 0.41320526599884033), ('franco', 0.36578550934791565), ('arriver', 0.35291430354118347), ('assassine', 0.3393854796886444), ('avance', 0.33924439549446106), ('gendarmerie', 0.33481234312057495), ('cancer', 0.32876473665237427), ('jamais', 0.3215842843055725), ('niveau', 0.31487542390823364), ('demeter', 0.3132854700088501), ('condamner', 0.3108205199241638), ('coute', 0.30837568640708923), ('reussi', 0.2982267439365387), ('creee', 0.2957681715488434), ('savoir', 0.29512885212898254), ('agences', 0.29339849948883057), ('atteintes', 0.2889772057533264), ('oublions', 0.28509244322776794), ('materiel', 0.2766176462173462), ('horreur', 0.27591386437416077)]


In [71]:
result = model_YJ.wv.most_similar(positive=['societe'], topn=20)
print(result)

[('solidaire', 0.3697938323020935), ('epanouissement', 0.367585688829422), ('enrichis', 0.3561038672924042), ('reparation', 0.34013983607292175), ('reparer', 0.33930259943008423), ('attendre', 0.32666659355163574), ('chers', 0.3224840760231018), ('ecole', 0.3206920623779297), ('envie', 0.31748467683792114), ('choisir', 0.3158099055290222), ('montee', 0.31053388118743896), ('constance', 0.3091202974319458), ('maltraite', 0.3085594177246094), ('humanistes', 0.30800527334213257), ('choix', 0.30488860607147217), ('transformer', 0.30214542150497437), ('comprendre', 0.29967543482780457), ('amis', 0.2960008978843689), ('methodes', 0.28936856985092163), ('accompagnerons', 0.28737327456474304)]


In [72]:
result = model_YJ.wv.most_similar(positive=['pouvoir'], topn=20)
print(result)

[('dignement', 0.435383141040802), ('justicesociale', 0.3740353286266327), ('envie', 0.34540998935699463), ('bonheur', 0.328460156917572), ('vitale', 0.32686424255371094), ('existence', 0.2857414186000824), ('pluralite', 0.27971914410591125), ('proposent', 0.2742132246494293), ('terme', 0.2724866271018982), ('met', 0.27054286003112793), ('preservee', 0.26857852935791016), ('confort', 0.268564373254776), ('methode', 0.2659911513328552), ('toutes', 0.26437506079673767), ('logement', 0.2515505850315094), ('accessible', 0.24831213057041168), ('choisir', 0.24819940328598022), ('pedagogique', 0.24806515872478485), ('agirons', 0.24558590352535248), ('corps', 0.2452787458896637)]


In [73]:
result = model_YJ.wv.most_similar(positive=['avenir'], topn=20)
print(result)

[('catastrophe', 0.4195205271244049), ('futur', 0.3866874873638153), ('attendre', 0.3787013292312622), ('votez', 0.37297937273979187), ('detruisent', 0.34441709518432617), ('marchelookup', 0.34085723757743835), ('comportements', 0.32780930399894714), ('chers', 0.3198343515396118), ('urnes', 0.31029632687568665), ('greveclimat', 0.30989396572113037), ('renouvelables', 0.3075825870037079), ('slava', 0.3070782721042633), ('jeunesse', 0.3062070310115814), ('engageons', 0.3059183955192566), ('lancer', 0.3053864538669586), ('avis', 0.3023163974285126), ('repose', 0.3014931380748749), ('planete', 0.3012007176876068), ('avril', 0.29625675082206726), ('marcheclimat', 0.295428603887558)]


In [74]:
result = model_YJ.wv.most_similar(positive=['histoire'], topn=20)
print(result)

[('trahir', 0.5514658689498901), ('ecrire', 0.5051293969154358), ('page', 0.4587285816669464), ('car', 0.43723833560943604), ('redonnons', 0.4077140688896179), ('detestation', 0.39828118681907654), ('algerie', 0.3771030008792877), ('entendre', 0.3721543550491333), ('yemen', 0.3709878623485565), ('aimez', 0.37028542160987854), ('guerres', 0.3687134087085724), ('abandonne', 0.36319100856781006), ('sens', 0.3600442409515381), ('lecons', 0.35016492009162903), ('construire', 0.3465043604373932), ('devoir', 0.34468257427215576), ('aime', 0.34080979228019714), ('club', 0.3331570327281952), ('unanime', 0.32858192920684814), ('qatar', 0.32361820340156555)]


In [75]:
result = model_YJ.wv.most_similar(positive=['contre'], topn=20)
print(result)

[('discriminations', 0.4124332368373871), ('systematique', 0.38969099521636963), ('lutter', 0.3535251319408417), ('terrorisme', 0.349017471075058), ('evasion', 0.3433181047439575), ('empecher', 0.3392636775970459), ('memoire', 0.33610308170318604), ('quasi', 0.3104781210422516), ('opposant', 0.29990699887275696), ('mariage', 0.2980043590068817), ('lutterons', 0.2967458665370941), ('surendettement', 0.29668718576431274), ('menacee', 0.2926871180534363), ('fraude', 0.277480810880661), ('justicesociale', 0.2737583816051483), ('materiel', 0.2727004885673523), ('livraisons', 0.2700607180595398), ('violences', 0.2700318694114685), ('lutte', 0.26635661721229553), ('europarl_fr', 0.26481789350509644)]


In [76]:
result = model_YJ.wv.most_similar(positive=['faut'], topn=20)
print(result)

[('binarite', 0.3023883104324341), ('menacent', 0.27747777104377747), ('finit', 0.26235538721084595), ('oligarques', 0.257680743932724), ('medecins', 0.2539386749267578), ('questionspol', 0.25344258546829224), ('propose', 0.2515656352043152), ('contrat', 0.2455209493637085), ('economiques', 0.24193577468395233), ('continuons', 0.2397322803735733), ('bonus', 0.23360297083854675), ('tue', 0.231244295835495), ('corps', 0.23121780157089233), ('integrer', 0.23088382184505463), ('polluantes', 0.230727881193161), ('paieront', 0.23063309490680695), ('choix', 0.22473084926605225), ('attaquent', 0.22175471484661102), ('russes', 0.21931172907352448), ('tout', 0.2191367894411087)]


In [77]:
result = model_YJ.wv.most_similar(positive=['crise'], topn=20)
print(result)

[('profiteurs', 0.5742793679237366), ('sanitaire', 0.5419210195541382), ('fortune', 0.4554649293422699), ('jaunes', 0.39989903569221497), ('consequences', 0.3900023102760315), ('migratoire', 0.38087356090545654), ('verse', 0.37539684772491455), ('gilets', 0.371444970369339), ('responsabilite', 0.36935338377952576), ('baisse', 0.36245405673980713), ('levee', 0.3621808886528015), ('pauvres', 0.36150923371315), ('impactees', 0.3605356812477112), ('covid', 0.3599587082862854), ('actuelle', 0.3559280335903168), ('imperieuse', 0.3547981381416321), ('payer', 0.3493860065937042), ('continent', 0.34800422191619873), ('euro', 0.33895519375801086), ('taxer', 0.336578369140625)]
